In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold, cross_validate
from sklearn.dummy import DummyClassifier
from numpy.random import RandomState
from sklearn.model_selection import KFold

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


In [ ]:
df = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)

In [ ]:

#remove columns Age, DAI, Rinab, IQ_T2, duration_meal, duration_play, Gender from df
df = df.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])
df


In [ ]:
df['label'] = df['label'].map({'no_rad': 0, 'rad': 1})

In [ ]:
# Reset the index
df = df.reset_index()

# Create the 'group' column and group by it
df['group'] = df['s_id'].str.split('_').str[0]
df_grouped = df.groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df.index.name = 's_id'

In [ ]:
df

In [ ]:
df = df.groupby('group').mean()
df['label'] = df['label'].map({0: 'no_rad', 1: 'rad'})

In [ ]:
# Perform a grid search for each classifier
X = df.drop('label', axis=1)
y = df['label']

# Create a GroupKFold object
gkf = KFold(n_splits=5)
X_shuffled, y_shuffled = shuffle(X, y, random_state=0)
# Define the classifiers and their parameters
classifiers = [
('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]

# Perform a grid search for each classifier
for name, classifier, params in classifiers:
    pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
    grid_search = GridSearchCV(pipeline, params, cv=gkf)
    grid_search.fit(X_shuffled, y_shuffled)

    print(f'Best parameters for {name}: {grid_search.best_params_}')
    print(f'Best score for {name}: {grid_search.best_score_}')

    # Calculate the cross-validated F1 score, precision, and recall
    f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro')
    precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro')
    recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro')

    print(f'Cross-validated F1 score for {name}: {f1_scores.mean()}')
    print(f'Cross-validated precision for {name}: {precision_scores.mean()}')
    print(f'Cross-validated recall for {name}: {recall_scores.mean()}')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Create a dummy classifier
dummy = DummyClassifier(strategy='most_frequent')

# Create an SVC classifier with specific parameters
svc_rbf = SVC(kernel='linear', C=10)

# Initialize a RandomState
rs = RandomState(123)

# Initialize a GroupKFold
gkf = GroupKFold(n_splits=5)

# Define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average='macro'),
           'recall': make_scorer(recall_score, average='macro'),
           'f1': make_scorer(f1_score, average='macro')}

# Create pipelines
pipeline_dummy = Pipeline([('scaler', StandardScaler()), ('dummy', dummy)])
pipeline_svc = Pipeline([('scaler', StandardScaler()), ('svc', svc_rbf)])

# Perform 10 random shuffles
for i in range(10):
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=rs)

    # Calculate the cross-validated scores of the dummy classifier
    scores_dummy = cross_validate(pipeline_dummy, X_shuffled, y_shuffled, groups=groups_shuffled, cv=gkf, scoring=scoring)

    print(f'Cross-validated scores of dummy classifier for shuffle {i+1}:')
    for metric, scores in scores_dummy.items():
        print(f'{metric}: {scores.mean()}')

    # Calculate the cross-validated scores of the SVC classifier
    scores_svc = cross_validate(pipeline_svc, X_shuffled, y_shuffled, groups=groups_shuffled, cv=gkf, scoring=scoring)

    print(f'Cross-validated scores of SVC classifier for shuffle {i+1}:')
    for metric, scores in scores_svc.items():
        print(f'{metric}: {scores.mean()}')

In [ ]:
# Fit the SelectKBest method
features = list(df.columns)
selector.fit(X_shuffled, y_shuffled)

# Get the scores
scores = selector.scores_

# Create a list of tuples, each containing a feature and its score
feature_scores = list(zip(features, scores))

# Sort the list of tuples by the score, in descending order
sorted_feature_scores = sorted(feature_scores, key=lambda x: x[1], reverse=True)

# Get the names of the features, in order of their score
sorted_features = [feature for feature, score in sorted_feature_scores]

print('Features sorted by importance:', sorted_features)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
X = df.drop(['label','group'], axis=1)
y = df['label']
groups = df['group']


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = LogisticRegression()

# Initialize the selector
# Fit the model
model.fit(X_train, y_train)

# Get the coefficients of the model
coefficients = model.coef_[0]

# Get the absolute values of the coefficients
abs_coefficients = abs(coefficients)

# Create a DataFrame with the feature names and coefficients
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': abs_coefficients})

# Sort the DataFrame by the coefficients in descending order
feature_importances = feature_importances.sort_values('importance', ascending=False)

# Print the features in order of their importance
print(feature_importances)

In [ ]:
# Fit the model
model.fit(X_train, y_train)

# Get the coefficients of the model
coefficients = model.coef_[0]

# Create a DataFrame with the feature names and coefficients
feature_importances = pd.DataFrame({'feature': X.columns, 'coefficient': coefficients})

# Calculate the absolute values of the coefficients for the importance
feature_importances['importance'] = abs(feature_importances['coefficient'])

# Sort the DataFrame by the importance in descending order
feature_importances = feature_importances.sort_values('importance', ascending=False)

# Print the features in order of their importance, including the sign of the coefficients
print(feature_importances)

In [ ]:
import numpy as np

# Get the sign of the coefficients
feature_importances['coefficient'] = np.sign(feature_importances['coefficient'])

# Replace positive and negative values with '+' and '-'
feature_importances['coefficient'] = feature_importances['coefficient'].map({1: 'rad', -1: 'no_rad'})

In [ ]:
feature_importances

In [ ]:
feature_importances.to_csv('feature_importances_meal.csv')